In [1]:
import pandas as pd

In [2]:
import numpy as np

### ready data

#### cleanup data


##### get spx data

In [70]:
spx = pd.read_csv('idx.csv',parse_dates = ['DATE'])\
.drop(columns = 'Unnamed: 0').set_index(['ID','DATE'])\
.sort_index().dropna(subset = 'close')\
.xs('SPX Index', level = 'ID')

In [73]:
spxw = spx.resample('W').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
    })

In [75]:
spxw['ma10'] = spxw['close'].rolling(10).mean()

##### get universe data

In [57]:
df = pd.read_csv('rty_2014.csv', parse_dates = ['DATE']).drop(columns = 'Unnamed: 0').set_index(['ID','DATE']).sort_index()

df = df[~df.index.duplicated()]

df = df.dropna(subset = ['close'])

#### run checks, how many tickers, how many months per year

#### calculate daily stats - adv, turnover

In [58]:
df['adv20']  = df.groupby(['ID']).apply(lambda x: x['volume'].rolling(20).mean()).reset_index(level = 0, drop = True).rename('adv20')

In [59]:
df['turnover'] = df['volume']*df['close']

In [60]:
df['turnover20']  = df.groupby(['ID']).apply(lambda x: x['turnover'].rolling(20).mean()).reset_index(level = 0, drop = True).rename('turnover20')

In [61]:
df['bapct'] = df['baspd']/df['close']

In [62]:
df = df.reset_index(level = 'ID')

#### resample weekly

In [64]:
weekly_df = df.groupby('ID').resample('W').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum',
    'market_cap':'last',
    'bid':'last',
    'ask':'last',
    'baspd':'last',
    'adv20':'last',
    'turnover':'last',
    'turnover20':'last',
    'bapct':'last'
    
})

#### calculate weekly stats

In [77]:
# weekly stats not needed as of now, going to do it in backtrader

In [78]:
weekly_df.tail()

open  high   low  close    volume   market_cap  \
ID             DATE                                                         
ZYXI UW Equity 2025-01-12  8.09  8.24  7.77   7.83   78710.0  249350624.0   
               2025-01-19  7.85  8.00  7.50   7.68  139495.0  244573792.0   
               2025-01-26  7.76  8.42  7.75   8.36   92701.0  266228752.0   
               2025-02-02  8.29  8.45  7.72   7.84   87519.0  249669072.0   
               2025-02-09  7.72  8.10  7.30   7.66  129961.0  243936864.0   

                            bid   ask     baspd     adv20   turnover  \
ID             DATE                                                    
ZYXI UW Equity 2025-01-12  7.83  7.85  0.049796  33683.45  178249.95   
               2025-01-19  7.66  7.68  0.042845  34758.35  253839.36   
               2025-01-26  8.35  8.36  0.042235  24697.15  198290.84   
               2025-02-02  7.83  7.84  0.058212  22883.70  248339.84   
               2025-02-09  7.59  7.67  0.077548  23622.05  496705.04   

                            turnover20     bapct  
ID             DATE                               
ZYXI UW Equity 2025-01-12  275413.3735  0.006360  
               2025-01-19  280428.3555  0.005579  
               2025-01-26  197375.9355  0.005052  
               2025-02-02  182215.6025  0.007425  
               2025-02-09  186850.0955  0.010124

In [83]:
spxw.to_csv('spxw.csv')

In [84]:
weekly_df.to_csv('weeklyr2k.csv')